In [1]:
import numpy as np
import numpy.random as rnd
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from ipywidgets import widgets, interact, interactive, fixed, interact_manual

from IPython.display import display, clear_output
import string
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
alphabet = list(string.ascii_uppercase)
max_categories = len(alphabet)

In [3]:
# GUI elements
categories_slider = widgets.IntSlider(min=1, max=max_categories, description='Categories', continuous_update=False, value=4)
categories_slider.layout.width = 'auto'

uniform_button = widgets.Button(description='Uniform', tooltip='Set all sliders to a uniform distribution')
normal_button = widgets.ToggleButton(
    value=False,
    description='Normal',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Approximate a normal distribution',
    icon='times'
)
draw_button = widgets.Button(description='Draw', icon='fa-bar-chart', button_style='success')
shape_slider = widgets.FloatSlider(min=0., max=5., step=0.1, value=0., continuous_update=False, description='Shape', disabled=True) 
shape_slider.layout.visibility='hidden'

dists = ['Custom', 'Geom', 'Logser', 'Planck', 'Uniform', 'Zipf']
distMap = {'Geom' : lambda shape: stats.geom.pmf(list(np.arange(start=1., stop=len(sliders)+1, step=1.)), shape), #Shape: 'p'
           'Logser' : lambda shape: stats.logser.pmf(list(np.arange(start=1., stop=len(sliders)+1, step=1.)), shape), #Shape: 'p'
           'Planck' : lambda shape: stats.planck.pmf(list(np.arange(start=0., stop=len(sliders), step=1.)), shape), #Shape: 'lambda'
           'Uniform' : lambda _: stats.randint.pmf(list(np.arange(start=0., stop=len(sliders), step=1.)), 0, categories_slider.value),
          'Zipf' : lambda shape: stats.zipf.pmf(list(np.arange(start=1., stop=len(sliders)+1, step=1.)), shape)} #Shape: 'a'
dropdown = widgets.Dropdown(options=dists, value='Custom', description='Distribution:', disabled=False)

sliders = []
texts = []

box = widgets.VBox()

In [4]:
out = widgets.Output()
out.layout.align_items = 'center'
fig = plt.figure()

In [5]:
def plot(x_axis, y_axis):
    with sns.axes_style("darkgrid"):
        fig.clear()
        ax = fig.add_subplot(1, 1, 1)
        ax.set_xlabel('Category')
        ax.set_ylabel('%')
        ax.set_ylim((0,100))
        sns.barplot(x=x_axis, y=y_axis, ax=ax, palette='pastel')
        fig.canvas.draw_idle()
    with out:
        clear_output()
        display(fig)

In [6]:
def cat_update(c_slider):
    categories = alphabet[0:categories_slider.value]
    probabilities = np.repeat(1./len(categories), categories_slider.value)
    
    plt.clf()
    plt.xlabel('Category')
    sns.barplot(x=pd.Series(categories), y=pd.Series(np.repeat(1, len(categories))), palette='Blues_d')
    plt.show()

In [7]:
def toggle_labels(change):

    if(normal_button.value):
        for text in texts:
            normal_button.button_style = 'success'
            normal_button.icon = 'check'
            text.disabled = False
            text.layout.visibility = 'visible'
    
    else:
        for text in texts:
            normal_button.button_style = 'danger'
            normal_button.icon = 'times'
            text.disabled = True
            text.layout.visibility = 'hidden'
        

In [8]:
def update_dist(change):

    values = distMap[dropdown.value](shape_slider.value)
    
    values_series=pd.Series(values)
    if(len(values_series.dropna())==0):
        values = np.repeat(0, len(values))

    for slider in sliders:

        index = sliders.index(slider)

        # If all IntTexts are within the correct range, use that order
        if [t.value in range(1,len(sliders)) for t in texts]:
            index = texts[sliders.index(slider)].value-1

        slider.value = int(np.round(values[index]*100, 0))

In [9]:
def update_gui():
    global sliders
    global box
    global texts
    
    # If number of categories have been updated, either remove or add more sliders
    diff = categories_slider.value - len(sliders)

    if(diff > 0):
        for i in range(len(sliders), len(sliders)+diff):
            desc = alphabet[i]
            sliders.append(widgets.IntSlider(min=0, max=100, value=50, description=desc, continuous_update=False, orientation='vertical'))
            
            texts.append(widgets.IntText(
            value=i+1,
            description='1-'+str(categories_slider.value),
            disabled=False,
            max=len(sliders),
        ))
    elif(diff < 0):
        sliders = sliders[0:categories_slider.value]
        texts = texts[0:categories_slider.value]

    # Add a label to each slider element
    pair_boxes = []
    
    for i in range(1, len(sliders)+1):
        texts[i-1].layout.width='150px'
        vbox = widgets.VBox([sliders[i-1], texts[i-1]])
        vbox.layout.align_items='flex-end'
        pair_boxes.append(vbox)

     # Put the layout together
    slider_box = widgets.HBox(sliders)
    dist_box = widgets.HBox([dropdown])
    box = widgets.VBox([dist_box, widgets.HBox(pair_boxes), shape_slider, draw_button])
    box.layout.align_items='center'
    

In [10]:
def uniform_action(change):
    
    value = distMap['Uniform'](range(0, categories_slider.value), None)
    
    for slider in sliders:
        slider.value = value[sliders.index(slider)]

In [11]:
def dropdown_action(change):
    shape_slider.hold_trait_notifications()

    # Update the shape slider according to the chosen distribution
    if(dropdown.value is 'Uniform' or dropdown.value is 'Custom'):
        shape_slider.disabled=True
        shape_slider.layout.visibility='hidden'
    else:

        if(dropdown.value is 'Geom'):
            shape_slider.min = 0.
            shape_slider.max = 1.
            shape_slider.step = 0.01
            shape_slider.value = 0.5
        elif(dropdown.value is 'Logser'):
            shape_slider.step = 0.01
            shape_slider.min = 0. + shape_slider.step
            shape_slider.max = 1. - shape_slider.step
            shape_slider.value = 0.6
        elif(dropdown.value is 'Planck'):
            shape_slider.min = 0.
            shape_slider.max = 1.
            shape_slider.step = 0.01
            shape_slider.value = 0.51
        elif(dropdown.value is 'Zipf'):
            shape_slider.max = 10.
            shape_slider.step = 0.01
            shape_slider.min = 0. + shape_slider.step
            shape_slider.value = 1.5

        shape_slider.disabled=False
        shape_slider.layout.visibility='visible'

    if(dropdown.value!='Custom'):
        update_dist(None)


In [12]:
def sorted_sliders():
    # Sort according to value
    s = list(sliders)
    
    # Weights set, use that order, otherwise alphabetical
    if [t.value in range(1,len(sliders)) for t in texts]:
        zippy = zip([slider for slider in sliders], [text.value for text in texts])
        zippy.sort(key= lambda (slider,weight): weight)
        s = [slider for (slider,weight) in zippy]
    else:    
        s.sort(key=lambda x: x.description, reverse=False)
        
    s.sort(key=lambda x: x.value, reverse=True)
    return s

In [13]:
def draw_graph(change):
    # Normalize the slider values
    s = sorted_sliders()
    probabilities = pd.Series([slider.value for slider in s])
    normalize_by = probabilities.sum()
    
    # Remove old GUI and show new
    clear_output()
    display(box, out)
    
    # Plot new graph
    plot(pd.Series([str(slider.description) for slider in s]), (probabilities/normalize_by)*100)

In [14]:
def draw(change):
    # Layout update
    update_gui()
    # Set labels visible/invisible
    toggle_labels(None)

    # Calculate slider values unless 'custom' is chosen
    if(dropdown.value != 'Custom'):
        update_dist(None)
    
    draw_graph(None)
  

In [15]:
interact(cat_update, c_slider=categories_slider);

interactive(children=(IntSlider(value=4, continuous_update=False, description=u'Categories', layout=Layout(wid…

In [16]:
draw_button.on_click(draw)
dropdown.observe(dropdown_action)

shape_slider.observe(draw)

draw(None)

Output(layout=Layout(align_items=u'center'), outputs=({u'output_type': u'display_data', u'data': {u'image/png'…